In [280]:
import pyblp
import numpy as np
import pandas as pd
import statsmodels.formula.api as smf

pyblp.options.digits = 3
pyblp.options.verbose = False
pd.options.display.precision = 3
pd.options.display.max_columns = 50

import IPython.display
IPython.display.display(IPython.display.HTML('<style>pre { white-space: pre !important; }</style>'))

import functions as fun

https://github.com/Mixtape-Sessions/Demand-Estimation

# 1. Import the data

In [281]:
product_data = pd.read_csv('Dataset.csv')
product_data = product_data[product_data['Market share'] != 0]
product_data


,ID,Year,Market share,Manufacturer,Model,Range,Price,HP,Chargetime,Type,Segment,Country,Sales
8,1,2021,1.037e-02,Aiways,U5,400,284621.700,201,34,SUV,C,CN,257
9,1,2022,5.976e-03,Aiways,U5,400,313681.829,201,34,SUV,C,CN,183
10,1,2023,2.860e-03,Aiways,U5,400,264524.000,201,34,SUV,C,CN,177
21,2,2023,4.848e-05,Aiways,U6,405,360638.000,214,34,SUV,C,CN,3
28,3,2019,4.063e-02,Audi,e-tron,375,979704.475,402,17,SUV,F,DE,222
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1154,189,2023,6.464e-05,Volvo,EX30,475,368245.000,268,28,SUV,B,SE,4
1163,190,2021,1.465e-02,Volvo,XC40,457,462060.600,402,28,SUV,C,SE,363
1164,190,2022,3.331e-02,Volvo,XC40,457,416263.400,402,28,SUV,C,SE,1020
1165,190,2023,3.175e-02,Volvo,XC40,457,439266.600,402,28,SUV,C,SE,1965


In [282]:
product_data['Price'] = product_data['Price']/10_000 #(Change in ms(%) for change in pris in 10.000)
product_data['HP'] = product_data['HP']/10           #(Change in ms(%) for change in HP in 10)
product_data['Range'] = product_data['Range']/10     #(Change in ms(%) for change in rækkevidde in 10)
# Creating dummy for china
product_data['China'] = (product_data['Country'] == 'CN').astype(int)

In [283]:
fun.BLP(product_data, 'HP')
fun.BLP(product_data, 'Chargetime')
fun.BLP(product_data, 'Range')

,ID,Year,Market share,Manufacturer,Model,Range,Price,HP,Chargetime,Type,Segment,Country,Sales,China,HP_sum,Chargetime_sum,Range_sum
8,1,2021,1.037e-02,Aiways,U5,40.0,28.462,20.1,34,SUV,C,CN,257,1,1358.3,1861,2151.3
9,1,2022,5.976e-03,Aiways,U5,40.0,31.368,20.1,34,SUV,C,CN,183,1,2033.9,2664,3218.0
10,1,2023,2.860e-03,Aiways,U5,40.0,26.452,20.1,34,SUV,C,CN,177,1,2640.3,3241,4177.4
21,2,2023,4.848e-05,Aiways,U6,40.5,36.064,21.4,34,SUV,C,CN,3,1,2639.0,3241,4176.9
28,3,2019,4.063e-02,Audi,e-tron,37.5,97.970,40.2,17,SUV,F,DE,222,0,390.1,570,580.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1154,189,2023,6.464e-05,Volvo,EX30,47.5,36.825,26.8,28,SUV,B,SE,4,0,2633.6,3247,4169.9
1163,190,2021,1.465e-02,Volvo,XC40,45.7,46.206,40.2,28,SUV,C,SE,363,0,1338.2,1867,2145.6
1164,190,2022,3.331e-02,Volvo,XC40,45.7,41.626,40.2,28,SUV,C,SE,1020,0,2013.8,2670,3212.3
1165,190,2023,3.175e-02,Volvo,XC40,45.7,43.927,40.2,28,SUV,C,SE,1965,0,2620.2,3247,4171.7


In [284]:
# GH instruments
fun.create_instrument_localsum(product_data, 'HP', 0.5)
fun.create_instrument_localsum(product_data, 'Chargetime', 0.5)
fun.create_instrument_localsum(product_data, 'Range', 0.5)

,ID,Year,Market share,Manufacturer,Model,Range,Price,HP,Chargetime,Type,Segment,Country,Sales,China,HP_sum,Chargetime_sum,Range_sum,HP_instrument_localsum,Chargetime_instrument_localsum,Range_instrument_localsum
8,1,2021,1.037e-02,Aiways,U5,40.0,28.462,20.1,34,SUV,C,CN,257,1,1358.3,1861,2151.3,740.5,1317,1499.9
9,1,2022,5.976e-03,Aiways,U5,40.0,31.368,20.1,34,SUV,C,CN,183,1,2033.9,2664,3218.0,1060.2,1796,2153.5
10,1,2023,2.860e-03,Aiways,U5,40.0,26.452,20.1,34,SUV,C,CN,177,1,2640.3,3241,4177.4,1501.0,2099,2792.3
21,2,2023,4.848e-05,Aiways,U6,40.5,36.064,21.4,34,SUV,C,CN,3,1,2639.0,3241,4176.9,1456.2,2099,2746.8
28,3,2019,4.063e-02,Audi,e-tron,37.5,97.970,40.2,17,SUV,F,DE,222,0,390.1,570,580.9,310.5,533,470.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1154,189,2023,6.464e-05,Volvo,EX30,47.5,36.825,26.8,28,SUV,B,SE,4,0,2633.6,3247,4169.9,1777.6,1813,2189.4
1163,190,2021,1.465e-02,Volvo,XC40,45.7,46.206,40.2,28,SUV,C,SE,363,0,1338.2,1867,2145.6,1026.5,1138,1169.9
1164,190,2022,3.331e-02,Volvo,XC40,45.7,41.626,40.2,28,SUV,C,SE,1020,0,2013.8,2670,3212.3,1621.7,1483,1702.8
1165,190,2023,3.175e-02,Volvo,XC40,45.7,43.927,40.2,28,SUV,C,SE,1965,0,2620.2,3247,4171.7,1978.6,1813,2206.7


# 2. Describe the data

In [285]:
product_data.sample(n=5, random_state=0)

,ID,Year,Market share,Manufacturer,Model,Range,Price,HP,Chargetime,Type,Segment,Country,Sales,China,HP_sum,Chargetime_sum,Range_sum,HP_instrument_localsum,Chargetime_instrument_localsum,Range_instrument_localsum
669,119,2022,5.584e-03,MG,Marvel R,38.8,34.999,17.7,38,SUV,C,CN,171,1,2036.3,2660,3219.2,1256.6,1903,2402.7
327,64,2021,8.880e-04,Honda,E,21.1,26.285,15.1,36,Hatchback,B,JP,22,0,1363.3,1859,2170.2,827.3,1284,1979.3
65,6,2023,7.255e-03,Audi,Q8 e-tron,51.8,79.697,33.5,27,SUV,E,DE,449,0,2626.9,3248,4165.6,1764.4,1879,2734.7
876,149,2020,1.413e-04,Polestar,2,51.3,45.363,41.5,28,Liftback,D,SE,2,0,712.6,1128,1134.5,518.1,821,927.5
769,132,2023,1.503e-03,Nissan,Leaf,32.8,25.464,14.7,43,Hatchback,C,JP,93,0,2645.7,3232,4184.6,1789.6,2141,3341.4


In [286]:
product_data['outside_share'] = 1 - product_data.groupby('Year')['Market share'].transform('sum')
product_data[['Market share', 'outside_share']].describe()

,Market share,outside_share
count,3.340e+02,3.340e+02
mean,3.293e-02,2.774e-15
std,8.309e-02,1.374e-15
min,1.616e-05,2.220e-16
25%,1.116e-03,1.554e-15
50%,7.540e-03,3.886e-15
75%,2.611e-02,3.886e-15
max,6.913e-01,3.886e-15


# 3. Estimate the pure logit model with OLS

In [287]:
product_data['logit_delta'] = np.log(product_data['Market share'] / product_data['outside_share'])
statsmodels_ols = smf.ols('logit_delta ~ 1 + Price + Range + HP + Chargetime + China', product_data)
statsmodels_results = statsmodels_ols.fit(cov_type='HC0')
statsmodels_results.summary2().tables[1]

,Coef.,Std.Err.,z,P>|z|,[0.025,0.975]
Intercept,27.291,0.891,30.623,5.982e-206,25.544,29.037
Price,-0.038,0.006,-6.886,5.756e-12,-0.049,-0.027
Range,0.068,0.017,3.918,8.919e-05,0.034,0.101
HP,0.020,0.015,1.398,1.622e-01,-0.008,0.049
Chargetime,0.004,0.017,0.246,8.056e-01,-0.029,0.037
China,-3.032,0.406,-7.460,8.631e-14,-3.829,-2.236


# 4. Run the same regression with PyBLP

In [288]:
product_data = product_data.rename(columns={
    'Year': 'market_ids',
    'Model': 'product_ids',
    'Market share': 'shares',
    'Price': 'prices',
})
product_data['demand_instruments0'] = product_data['prices']
ols_problem = pyblp.Problem(pyblp.Formulation('1 + prices + Range + HP + Chargetime + China'), product_data)
ols_problem

Dimensions:
 T    N    K1    MD 
---  ---  ----  ----
11   334   6     6  

Formulations:
     Column Indices:         0     1       2     3       4         5  
--------------------------  ---  ------  -----  ---  ----------  -----
X1: Linear Characteristics   1   prices  Range  HP   Chargetime  China

In [289]:
pd.DataFrame(ols_problem.products.ZD).sample(n=5, random_state=0)

,0,1,2,3,4,5
166,34.999,1.0,38.8,17.7,38.0,1.0
78,26.285,1.0,21.1,15.1,36.0,0.0
15,79.697,1.0,51.8,33.5,27.0,0.0
221,45.363,1.0,51.3,41.5,28.0,0.0
194,25.464,1.0,32.8,14.7,43.0,0.0


In [290]:
ols_results = ols_problem.solve(method='1s')
ols_results

Problem Results Summary:
GMM   Objective  Clipped  Weighting Matrix  Covariance Matrix
Step    Value    Shares   Condition Number  Condition Number 
----  ---------  -------  ----------------  -----------------
 1    +4.77E-22     0        +2.02E+05          +3.52E+05    

Cumulative Statistics:
Computation   Objective 
   Time      Evaluations
-----------  -----------
 00:00:00         1     

Beta Estimates (Robust SEs in Parentheses):
     1         prices        Range         HP       Chargetime      China   
-----------  -----------  -----------  -----------  -----------  -----------
 +2.73E+01    -3.83E-02    +6.85E-02    +2.02E-02    +4.15E-03    -3.02E+00 
(+8.90E-01)  (+5.55E-03)  (+1.72E-02)  (+1.47E-02)  (+1.68E-02)  (+4.04E-01)

In [291]:
pd.DataFrame(index=ols_results.beta_labels, data={
    ("Estimates", "Statsmodels"): statsmodels_results.params.values,
    ("Estimates", "PyBLP"): ols_results.beta.flat,
    ("SEs", "Statsmodels"): statsmodels_results.bse.values,
    ("SEs", "PyBLP"): ols_results.beta_se.flat,
})

Estimates                 SEs       
           Statsmodels   PyBLP Statsmodels  PyBLP
1               27.291  27.275       0.891  0.890
prices          -0.038  -0.038       0.006  0.006
Range            0.068   0.069       0.017  0.017
HP               0.020   0.020       0.015  0.015
Chargetime       0.004   0.004       0.017  0.017
China           -3.032  -3.016       0.406  0.404

# 5. Add market and product fixed effects

In [292]:
fe_problem = pyblp.Problem(pyblp.Formulation('0 + prices + Range', absorb='C(market_ids) + C(product_ids)'), product_data)
fe_problem

Dimensions:
 T    N    K1    MD    ED 
---  ---  ----  ----  ----
11   334   2     2     2  

Formulations:
     Column Indices:          0       1  
--------------------------  ------  -----
X1: Linear Characteristics  prices  Range

In [293]:
fe_results = fe_problem.solve(method='1s')
fe_results

Detected that the 2SLS weighting matrix is nearly singular with condition number +1.12E+19. To disable singularity checks, set options.singular_tol = numpy.inf.
Detected that the estimated covariance matrix of aggregate GMM moments is nearly singular with condition number +7.50E+18. To disable singularity checks, set options.singular_tol = numpy.inf.
Detected that the estimated covariance matrix of aggregate GMM moments is nearly singular with condition number +7.50E+18. To disable singularity checks, set options.singular_tol = numpy.inf.


Problem Results Summary:
GMM   Objective  Clipped  Weighting Matrix  Covariance Matrix
Step    Value    Shares   Condition Number  Condition Number 
----  ---------  -------  ----------------  -----------------
 1    +1.03E-14     0        +1.12E+19            +INF       

Cumulative Statistics:
Computation   Objective 
   Time      Evaluations
-----------  -----------
 00:00:00         1     

Beta Estimates (Robust SEs in Parentheses):
  prices        Range   
-----------  -----------
 -2.17E-02    +2.18E-20 
(+1.53E-02)  (+1.54E-20)

# 6. Add an instrument for price

# 6.1. BLP

In [294]:
first_stage1 = smf.ols('prices ~ 0 + HP_sum + Range_sum + Chargetime_sum + C(market_ids) + C(product_ids)', product_data)
first_stage_results1 = first_stage1.fit(cov_type='HC0')
first_stage_results1.summary2().tables[1].sort_index(ascending=False)

,Coef.,Std.Err.,z,P>|z|,[0.025,0.975]
Range_sum,-0.046,0.020,-2.235,2.541e-02,-0.086,-0.006
HP_sum,-0.212,0.043,-4.948,7.502e-07,-0.297,-0.128
Chargetime_sum,0.237,0.014,16.693,1.463e-62,0.209,0.265
C(product_ids)[T.ë-C4 X],-8.459,0.606,-13.949,3.172e-44,-9.647,-7.270
C(product_ids)[T.up!],-9.830,1.029,-9.557,1.214e-21,-11.846,-7.814
...,...,...,...,...,...,...
C(market_ids)[2017],27.689,1.766,15.681,2.053e-55,24.228,31.150
C(market_ids)[2016],26.708,1.782,14.985,9.198e-51,23.214,30.201
C(market_ids)[2015],12.291,2.140,5.745,9.218e-09,8.097,16.484
C(market_ids)[2014],14.485,3.067,4.723,2.323e-06,8.474,20.495


In [295]:
product_data = product_data.drop(columns='HP_sum').rename(columns={'price_instrument1': 'HP_sum'})
product_data = product_data.drop(columns='Range_sum').rename(columns={'price_instrument2': 'Range_sum'})
product_data = product_data.drop(columns='Chargetime_sum').rename(columns={'price_instrument2': 'Chargetime_sum'})
iv_problem1 = pyblp.Problem(pyblp.Formulation('0 + prices + Range', absorb='C(market_ids) + C(product_ids)'), product_data)
iv_problem1

Dimensions:
 T    N    K1    MD    ED 
---  ---  ----  ----  ----
11   334   2     2     2  

Formulations:
     Column Indices:          0       1  
--------------------------  ------  -----
X1: Linear Characteristics  prices  Range

In [296]:
iv_results1 = iv_problem1.solve(method='1s')
iv_results1

Detected that the 2SLS weighting matrix is nearly singular with condition number +1.12E+19. To disable singularity checks, set options.singular_tol = numpy.inf.
Detected that the estimated covariance matrix of aggregate GMM moments is nearly singular with condition number +7.50E+18. To disable singularity checks, set options.singular_tol = numpy.inf.
Detected that the estimated covariance matrix of aggregate GMM moments is nearly singular with condition number +7.50E+18. To disable singularity checks, set options.singular_tol = numpy.inf.


Problem Results Summary:
GMM   Objective  Clipped  Weighting Matrix  Covariance Matrix
Step    Value    Shares   Condition Number  Condition Number 
----  ---------  -------  ----------------  -----------------
 1    +1.03E-14     0        +1.12E+19            +INF       

Cumulative Statistics:
Computation   Objective 
   Time      Evaluations
-----------  -----------
 00:00:00         1     

Beta Estimates (Robust SEs in Parentheses):
  prices        Range   
-----------  -----------
 -2.17E-02    +2.18E-20 
(+1.53E-02)  (+1.54E-20)

# 6.2. GH

In [297]:
first_stage2 = smf.ols('prices ~ 0 + HP_instrument_localsum + Range_instrument_localsum + Chargetime_instrument_localsum + C(market_ids) + C(product_ids)', product_data)
first_stage_results2 = first_stage2.fit(cov_type='HC0')
first_stage_results2.summary2().tables[1].sort_index(ascending=False)

,Coef.,Std.Err.,z,P>|z|,[0.025,0.975]
Range_instrument_localsum,5.510e-04,0.002,0.281,7.788e-01,-0.003,4.396e-03
HP_instrument_localsum,8.111e-03,0.006,1.409,1.589e-01,-0.003,1.939e-02
Chargetime_instrument_localsum,-6.219e-03,0.003,-2.265,2.353e-02,-0.012,-8.369e-04
C(product_ids)[T.ë-C4 X],-1.368e+01,2.454,-5.574,2.486e-08,-18.490,-8.870e+00
C(product_ids)[T.up!],-2.069e+01,2.781,-7.437,1.029e-13,-26.137,-1.523e+01
...,...,...,...,...,...,...
C(market_ids)[2017],4.162e+01,3.629,11.470,1.872e-30,34.508,4.873e+01
C(market_ids)[2016],4.007e+01,3.573,11.213,3.516e-29,33.064,4.707e+01
C(market_ids)[2015],3.526e+01,3.525,10.002,1.500e-23,28.349,4.217e+01
C(market_ids)[2014],3.337e+01,4.229,7.891,2.986e-15,25.082,4.166e+01


In [298]:
product_data = product_data.drop(columns='HP_instrument_localsum').rename(columns={'price_instrument1': 'HP_instrument_localsum'})
product_data = product_data.drop(columns='Range_instrument_localsum').rename(columns={'price_instrument2': 'Range_instrument_localsum'})
product_data = product_data.drop(columns='Chargetime_instrument_localsum').rename(columns={'price_instrument2': 'Chargetime_instrument_localsum'})
iv_problem2 = pyblp.Problem(pyblp.Formulation('0 + prices + Range', absorb='C(market_ids) + C(product_ids)'), product_data)
iv_problem2

Dimensions:
 T    N    K1    MD    ED 
---  ---  ----  ----  ----
11   334   2     2     2  

Formulations:
     Column Indices:          0       1  
--------------------------  ------  -----
X1: Linear Characteristics  prices  Range

In [299]:
iv_results2 = iv_problem1.solve(method='1s')
iv_results2

Detected that the 2SLS weighting matrix is nearly singular with condition number +1.12E+19. To disable singularity checks, set options.singular_tol = numpy.inf.
Detected that the estimated covariance matrix of aggregate GMM moments is nearly singular with condition number +7.50E+18. To disable singularity checks, set options.singular_tol = numpy.inf.
Detected that the estimated covariance matrix of aggregate GMM moments is nearly singular with condition number +7.50E+18. To disable singularity checks, set options.singular_tol = numpy.inf.


Problem Results Summary:
GMM   Objective  Clipped  Weighting Matrix  Covariance Matrix
Step    Value    Shares   Condition Number  Condition Number 
----  ---------  -------  ----------------  -----------------
 1    +1.03E-14     0        +1.12E+19            +INF       

Cumulative Statistics:
Computation   Objective 
   Time      Evaluations
-----------  -----------
 00:00:00         1     

Beta Estimates (Robust SEs in Parentheses):
  prices        Range   
-----------  -----------
 -2.17E-02    +2.18E-20 
(+1.53E-02)  (+1.54E-20)

In [300]:
pd.DataFrame(index=fe_results.beta_labels, data={
    ("Estimates", "OLS"): ols_results.beta[1:3].flat,
    ("Estimates", "+FE"): fe_results.beta.flat,
    ("Estimates", "+IV_BLP"): iv_results1.beta.flat,
    ("Estimates", "+IV_GH"): iv_results2.beta.flat,
    ("SEs", "OLS"): ols_results.beta_se[1:3].flat,
    ("SEs", "+FE"): fe_results.beta_se.flat,
    ("SEs", "+IV_BLP"): iv_results1.beta_se.flat,
    ("SEs", "+IV_GH"): iv_results2.beta_se.flat,
})

Estimates                                     SEs             \
             OLS        +FE    +IV_BLP     +IV_GH    OLS        +FE   
prices    -0.038 -2.171e-02 -2.171e-02 -2.171e-02  0.006  1.530e-02   
Range      0.069  2.181e-20  2.181e-20  2.181e-20  0.017  1.537e-20   

                              
          +IV_BLP     +IV_GH  
prices  1.530e-02  1.530e-02  
Range   1.537e-20  1.537e-20

# 7. Decrease price of Chinese cars see what happens


In [301]:
counterfactual_market = 2023
counterfactual_data = product_data.loc[product_data['market_ids'] == counterfactual_market, ['product_ids', 'prices', 'Range', 'HP', 'Chargetime', 'shares']]
counterfactual_data

,product_ids,prices,Range,HP,Chargetime,shares
10,U5,26.452,40.0,20.1,34,2.860e-03
21,U6,36.064,40.5,21.4,34,4.848e-05
32,e-tron,67.304,37.5,40.2,17,1.099e-03
43,e-tron GT,121.934,47.2,52.2,17,3.717e-04
54,Q4 e-tron,65.855,49.6,28.1,28,3.796e-02
...,...,...,...,...,...,...
1132,up!,17.485,25.6,8.1,48,1.277e-03
1143,C40,43.066,46.6,40.2,28,8.354e-03
1154,EX30,36.825,47.5,26.8,28,6.464e-05
1165,XC40,43.927,45.7,40.2,28,3.175e-02


In [302]:
counterfactual_data['new_prices'] = counterfactual_data['prices']*0.8
#counterfactual_data.loc[counterfactual_data['product_ids'] == 'F1B04', 'new_prices'] /= 2
counterfactual_data['new_shares'] = iv_results2.compute_shares(market_id=counterfactual_market, prices=counterfactual_data['new_prices'])
counterfactual_data['iv_change'] = 100 * (counterfactual_data['new_shares'] - counterfactual_data['shares']) / counterfactual_data['shares']
counterfactual_data

,product_ids,prices,Range,HP,Chargetime,shares,new_prices,new_shares,iv_change
10,U5,26.452,40.0,20.1,34,2.860e-03,21.162,2.649e-03,-7.384
21,U6,36.064,40.5,21.4,34,4.848e-05,28.851,4.681e-05,-3.437
32,e-tron,67.304,37.5,40.2,17,1.099e-03,53.843,1.215e-03,10.594
43,e-tron GT,121.934,47.2,52.2,17,3.717e-04,97.547,5.211e-04,40.207
54,Q4 e-tron,65.855,49.6,28.1,28,3.796e-02,52.684,4.172e-02,9.901
...,...,...,...,...,...,...,...,...,...
1132,up!,17.485,25.6,8.1,48,1.277e-03,13.988,1.137e-03,-10.922
1143,C40,43.066,46.6,40.2,28,8.354e-03,34.453,8.316e-03,-0.455
1154,EX30,36.825,47.5,26.8,28,6.464e-05,29.460,6.262e-05,-3.117
1165,XC40,43.927,45.7,40.2,28,3.175e-02,35.141,3.173e-02,-0.082


# 8. Compute demand elasticities


In [303]:
iv_elasticities = iv_results2.compute_elasticities(market_id=counterfactual_market)
pd.DataFrame(iv_elasticities)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,...,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99
0,-0.573,3.796e-05,0.002,9.840e-04,0.054,0.013,1.112e-05,0.034,0.009,0.003,0.014,0.017,0.017,0.002,0.002,6.215e-05,3.600e-04,9.827e-05,2.982e-04,0.002,0.002,6.494e-04,1.004e-04,0.015,5.022e-04,...,0.007,0.014,2.674e-04,0.047,2.215e-05,4.004e-05,6.633e-04,0.053,0.004,0.003,0.255,0.014,1.036e-04,2.917e-04,1.097e-05,0.014,0.037,0.014,0.003,0.022,4.847e-04,0.008,5.168e-05,0.030,1.417e-04
1,0.002,-7.831e-01,0.002,9.840e-04,0.054,0.013,1.112e-05,0.034,0.009,0.003,0.014,0.017,0.017,0.002,0.002,6.215e-05,3.600e-04,9.827e-05,2.982e-04,0.002,0.002,6.494e-04,1.004e-04,0.015,5.022e-04,...,0.007,0.014,2.674e-04,0.047,2.215e-05,4.004e-05,6.633e-04,0.053,0.004,0.003,0.255,0.014,1.036e-04,2.917e-04,1.097e-05,0.014,0.037,0.014,0.003,0.022,4.847e-04,0.008,5.168e-05,0.030,1.417e-04
2,0.002,3.796e-05,-1.460,9.840e-04,0.054,0.013,1.112e-05,0.034,0.009,0.003,0.014,0.017,0.017,0.002,0.002,6.215e-05,3.600e-04,9.827e-05,2.982e-04,0.002,0.002,6.494e-04,1.004e-04,0.015,5.022e-04,...,0.007,0.014,2.674e-04,0.047,2.215e-05,4.004e-05,6.633e-04,0.053,0.004,0.003,0.255,0.014,1.036e-04,2.917e-04,1.097e-05,0.014,0.037,0.014,0.003,0.022,4.847e-04,0.008,5.168e-05,0.030,1.417e-04
3,0.002,3.796e-05,0.002,-2.647e+00,0.054,0.013,1.112e-05,0.034,0.009,0.003,0.014,0.017,0.017,0.002,0.002,6.215e-05,3.600e-04,9.827e-05,2.982e-04,0.002,0.002,6.494e-04,1.004e-04,0.015,5.022e-04,...,0.007,0.014,2.674e-04,0.047,2.215e-05,4.004e-05,6.633e-04,0.053,0.004,0.003,0.255,0.014,1.036e-04,2.917e-04,1.097e-05,0.014,0.037,0.014,0.003,0.022,4.847e-04,0.008,5.168e-05,0.030,1.417e-04
4,0.002,3.796e-05,0.002,9.840e-04,-1.376,0.013,1.112e-05,0.034,0.009,0.003,0.014,0.017,0.017,0.002,0.002,6.215e-05,3.600e-04,9.827e-05,2.982e-04,0.002,0.002,6.494e-04,1.004e-04,0.015,5.022e-04,...,0.007,0.014,2.674e-04,0.047,2.215e-05,4.004e-05,6.633e-04,0.053,0.004,0.003,0.255,0.014,1.036e-04,2.917e-04,1.097e-05,0.014,0.037,0.014,0.003,0.022,4.847e-04,0.008,5.168e-05,0.030,1.417e-04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,0.002,3.796e-05,0.002,9.840e-04,0.054,0.013,1.112e-05,0.034,0.009,0.003,0.014,0.017,0.017,0.002,0.002,6.215e-05,3.600e-04,9.827e-05,2.982e-04,0.002,0.002,6.494e-04,1.004e-04,0.015,5.022e-04,...,0.007,0.014,2.674e-04,0.047,2.215e-05,4.004e-05,6.633e-04,0.053,0.004,0.003,0.255,0.014,1.036e-04,2.917e-04,1.097e-05,0.014,0.037,0.014,0.003,0.022,-3.792e-01,0.008,5.168e-05,0.030,1.417e-04
96,0.002,3.796e-05,0.002,9.840e-04,0.054,0.013,1.112e-05,0.034,0.009,0.003,0.014,0.017,0.017,0.002,0.002,6.215e-05,3.600e-04,9.827e-05,2.982e-04,0.002,0.002,6.494e-04,1.004e-04,0.015,5.022e-04,...,0.007,0.014,2.674e-04,0.047,2.215e-05,4.004e-05,6.633e-04,0.053,0.004,0.003,0.255,0.014,1.036e-04,2.917e-04,1.097e-05,0.014,0.037,0.014,0.003,0.022,4.847e-04,-0.927,5.168e-05,0.030,1.417e-04
97,0.002,3.796e-05,0.002,9.840e-04,0.054,0.013,1.112e-05,0.034,0.009,0.003,0.014,0.017,0.017,0.002,0.002,6.215e-05,3.600e-04,9.827e-05,2.982e-04,0.002,0.002,6.494e-04,1.004e-04,0.015,5.022e-04,...,0.007,0.014,2.674e-04,0.047,2.215e-05,4.004e-05,6.633e-04,0.053,0.004,0.003,0.255,0.014,1.036e-04,2.917e-04,1.097e-05,0.014,0.037,0.014,0.003,0.022,4.847e-04,0.008,-7.996e-01,0.030,1.417e-04
98,0.002,3.796e-05,0.002,9.840e-04,0.054,0.013,1.112e-05,0.034,0.009,0.003,0.014,0.017,0.017,0.002,0.002,6.215e-05,3.600e-04,9.827e-05,2.982e-04,0.002,0.002,6.494e-04,1.004e-04,0.015,5.022e-04,...,0.007,0.014,2.674e-04,0.047,2.215e-05,4.004e-05,6.633e-04,0.053,0.004,0.003,0.255,0.014,1.036e-04,2.917e-04,1.097e-05,0.014,0.037,0.014,0.003,0.022,4.847e-04,0.008,5.168e-05,-0.924,1.417e-04
